In [1]:
# import project directory helper
import os, sys, inspect
currentDir = os.getcwd()
parentDir = os.path.dirname(currentDir)
sys.path.insert(0, parentDir)

In [2]:
# import pyspark packages
from pyspark import SparkContext as sc
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession, functions, types
from pyspark.sql.types import *

In [3]:
# import util packages
from IPython.display import display
import pandas as pd

In [4]:
# sc = sc(appName="attraction")
# sqlContext = SQLContext(sc)
# import spark packages
spark = SparkSession.builder.appName("attraction").getOrCreate()

# tripadvisor_attr_href_cat

In [5]:
# define path
tripad_attr_path = parentDir + '/crawler/datasets/tripadvisor_dataset/attractions/'
parquet_path = parentDir + '/crawler/datasets/tripadvisor_dataset/spark/'

In [6]:
# read json as spark dataframe
attr_href_cat_spark_df = spark.read.json(tripad_attr_path +
                                         'tripadvisor_attr_href_cat.json')
# attr_href_cat_df = pd.read_json(tripadvisor_attr_href_cat, orient='records')

In [7]:
# write spark dataframe to parquet
attr_href_cat_spark_df.write.parquet(parquet_path +
                                     'tripadvisor_attr_href_cat',
                                     mode='overwrite')

In [8]:
# read spark dataframe from parquet
attr_href_cat_spark_df = spark.read.parquet(parquet_path + 'tripadvisor_attr_href_cat')

In [9]:
# convert spark dataframe to pandas dataframe
attr_href_cat_df = attr_href_cat_spark_df.toPandas()

In [10]:
# add activityId, locationId columns to dataframe
attr_href_cat_df['activityId'] = attr_href_cat_df['href'].str.extract(
    r'd(\d+)', expand=True)
attr_href_cat_df['locationId'] = attr_href_cat_df['href'].str.extract(
    r'g(\d+)', expand=True)

In [11]:
# rearrange pandas dataframe columns order
attr_href_cat_df_cl = ['activityId', 'locationId', 'href',
                       'category']  # columns order
attr_href_cat_df = attr_href_cat_df.reindex(columns=attr_href_cat_df_cl)

In [12]:
# visualize pandas dataframe
display(attr_href_cat_df.count())
display(attr_href_cat_df.head())

activityId    4257
locationId    4257
href          4260
category      4260
dtype: int64

,activityId,locationId,href,category
0,NaN,NaN,None,None
1,12962337,298570,/AttractionProductReview-g298570-d12962337-Mal...,"[featured_tours_and_tickets, luxury_special_oc..."
2,12905660,298570,/AttractionProductReview-g298570-d12905660-Pet...,"[featured_tours_and_tickets, sightseeing_ticke..."
3,11807013,298570,/AttractionProductReview-g298570-d11807013-Dis...,"[featured_tours_and_tickets, private_custom_to..."
4,12471375,298570,/AttractionProductReview-g298570-d12471375-Gen...,"[featured_tours_and_tickets, luxury_special_oc..."


In [13]:
# showing and removing duplicate value
print('Duplicated values:')
print(attr_href_cat_df[attr_href_cat_df['activityId'].duplicated(keep=False)])
print()
print('Length before removing duplicated values: ' +
      str(len(attr_href_cat_df)))

Duplicated values:
     activityId locationId                                               href  \
0           NaN        NaN                                               None   
376         NaN        NaN  /AttractionProductReview?partner=Viator&produc...   
1339        NaN        NaN  /AttractionProductReview?partner=Viator&produc...   
1945        NaN        NaN  /AttractionProductReview?partner=Viator&produc...   
4261        NaN        NaN                                               None   

                                       category  
0                                          None  
376    [classes_workshops, food_wine_nightlife]  
1339                       [outdoor_activities]  
1945  [private_custom_tours, tours_sightseeing]  
4261                                       None  

Length before removing duplicated values: 4262


In [14]:
# # dropping duplicate values
attr_href_cat_df = attr_href_cat_df.drop_duplicates(subset='activityId',
                                                    keep=False)
print('Length after removing duplicated values: ' + str(len(attr_href_cat_df)))

Length after removing duplicated values: 4257


In [15]:
# dropping nan values
attr_href_cat_df = attr_href_cat_df.dropna()
print('Length after removing NaN values: ' + str(len(attr_href_cat_df)))

Length after removing NaN values: 4257


In [16]:
# create spark dataframe from pandas dataframe
attr_href_cat_spark_df = spark.createDataFrame(attr_href_cat_df)

In [34]:
# visualize spark dataframe
display(attr_href_cat_spark_df.count())
attr_href_cat_spark_df.show()

4257

+----------+----------+--------------------+--------------------+
|activityId|locationId|                href|            category|
+----------+----------+--------------------+--------------------+
|  12962337|    298570|/AttractionProduc...|[featured_tours_a...|
|  12905660|    298570|/AttractionProduc...|[featured_tours_a...|
|  11807013|    298570|/AttractionProduc...|[featured_tours_a...|
|  12471375|    298570|/AttractionProduc...|[featured_tours_a...|
|  12469580|    298570|/AttractionProduc...|[featured_tours_a...|
|  16722683|    298570|/AttractionProduc...|[featured_tours_a...|
|  11454341|   1096277|/AttractionProduc...|[featured_tours_a...|
|  11812460|    298570|/AttractionProduc...|[featured_tours_a...|
|  11992275|   1096277|/AttractionProduc...|[featured_tours_a...|
|  11991123|    660694|/AttractionProduc...|[featured_tours_a...|
|  11482502|    298570|/AttractionProduc...|[featured_tours_a...|
|  19350513|    635527|/AttractionProduc...|[featured_tours_a...|
|  1147497

# tripad_attr_activity

In [18]:
# define path
tripad_attr_path = parentDir + '/crawler/datasets/tripadvisor_dataset/attractions/'
parquet_path = parentDir + '/crawler/datasets/tripadvisor_dataset/spark/'

In [19]:
# read json as spark dataframe
tripad_act_spark_df = spark.read.json(tripad_attr_path +
                                      'tripad_attr_activity.json')

In [20]:
# write spark dataframe to parquet
tripad_act_spark_df.write.parquet(parquet_path + 'tripad_attr_activity',
                                  mode='overwrite')

In [21]:
# read spark dataframe from parquet
tripad_act_spark_df = spark.read.parquet(parquet_path + 'tripad_attr_activity')

In [22]:
# drop unwanted columns
tripad_act_spark_df = tripad_act_spark_df.drop('__COMMENT')
tripad_act_spark_df = tripad_act_spark_df.dropna(subset='productHeader')

In [23]:
# select disired columns
tripad_act_spark_df = tripad_act_spark_df.select(
    tripad_act_spark_df.productHeader.activityId.alias("activityId"),
    tripad_act_spark_df.productHeader.productTitle.alias("name"),
    tripad_act_spark_df.tourPlanner.numericPrice.alias("price"),
    tripad_act_spark_df.productHeader.reviewRating.alias("rating"))

In [33]:
# visualize spark dataframe
display(tripad_act_spark_df.count())
tripad_act_spark_df.show()

4257

+----------+--------------------+------+------+
|activityId|                name| price|rating|
+----------+--------------------+------+------+
|  12962337|Malaysia Countrys...| 43.27|   4.5|
|  12905660|Petronas Twin Tow...|119.49|   4.0|
|  11807013|Discover the Best...|142.75|   4.5|
|  12471375|Genting Highlands...|103.01|   4.5|
|  12469580|Half-Day Kuala Lu...| 43.27|   4.0|
|  16722683|KL Hop On Hop Off...| 53.07|   3.0|
|  11454341|Langkawi Archipel...|515.06|   5.0|
|  11812460|Private Half-Day ...|377.67|   5.0|
|  11992275|12 Flights: Zipli...|473.86|   5.0|
|  11991123|Cheong Fatt Tze G...| 20.61|   4.5|
|  11482502|Batik Bag Paintin...|119.49|   5.0|
|  19350513|Chef Samuel's Pri...|700.48|   5.0|
|  11474972|Half-Day Penang C...|115.81|   5.0|
|  12648212|North Borneo Crui...|222.48|   4.5|
|  11470871|Hot Air Balloon F...|947.71|   5.0|
|  12468200|Private Tour: Kua...|407.98|   5.0|
|  12934938|Kinabatangan Rive...|568.63|   5.0|
|  11452959|Small-Group Food ...|243.14|

# tripad_attr_location

In [25]:
# define path
tripad_attr_path = parentDir + '/crawler/datasets/tripadvisor_dataset/attractions/'
parquet_path = parentDir + '/crawler/datasets/tripadvisor_dataset/spark/'

In [26]:
# read json as spark dataframe
tripad_loc_spark_df = spark.read.json(tripad_attr_path +
                                      'tripad_attr_location.json')

In [27]:
# write spark dataframe to parquet
tripad_loc_spark_df.write.parquet(parquet_path + 'tripad_attr_location',
                                  mode='overwrite')

In [28]:
# read spark dataframe from parquet
tripad_loc_spark_df = spark.read.parquet(parquet_path + 'tripad_attr_location')

In [29]:
# drop unwanted columns
tripad_loc_spark_df = tripad_loc_spark_df.drop('__COMMENT')
tripad_loc_spark_df = tripad_loc_spark_df.dropna(subset='location_id')

In [30]:
# select disired columns
tripad_loc_spark_df = tripad_loc_spark_df.select(
    tripad_loc_spark_df.location_id.alias("locationId"),
    tripad_loc_spark_df.name.alias("city"),
    tripad_loc_spark_df.ancestors[0]['name'].alias("state"),
    tripad_loc_spark_df.parent_display_name.alias("country"),
    tripad_loc_spark_df.latitude.alias("latitude"),
    tripad_loc_spark_df.longitude.alias("longitude"))

In [32]:
# visualize spark dataframe
display(tripad_loc_spark_df.count())
tripad_loc_spark_df.show()

98

+----------+--------------------+--------------------+-----------------+--------+---------+
|locationId|                city|               state|          country|latitude|longitude|
+----------+--------------------+--------------------+-----------------+--------+---------+
|    298570|        Kuala Lumpur| Wilayah Persekutuan|         Malaysia|3.156802|101.69717|
|   1096277|                Kuah|            Langkawi|         Malaysia|6.326672|99.843025|
|    660694|       Penang Island|              Penang|         Malaysia|5.388071|100.29352|
|    635527|      Batu Ferringhi|       Penang Island|         Malaysia|5.471783|100.24629|
|    298303|         George Town|       Penang Island|         Malaysia|5.411938|100.32664|
|    298307|       Kota Kinabalu|Kota Kinabalu Dis...|         Malaysia|5.979383|116.07349|
|    303997|            Sandakan|   Sandakan Division|         Malaysia|5.838944|118.11561|
|    298283|            Langkawi|   Langkawi District|         Malaysia|6.351074

In [35]:
# convert spark dataframe to pandas dataframe
tripad_loc_df = tripad_loc_spark_df.toPandas()

In [36]:
# visualize pandas dataframe
display(tripad_loc_df.count())
pd.set_option('display.max_rows', None)
display(tripad_loc_df)
pd.reset_option('display.max_rows')

locationId    98
city          98
state         97
country       98
latitude      97
longitude     97
dtype: int64

,locationId,city,state,country,latitude,longitude
0,298570,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717
1,1096277,Kuah,Langkawi,Malaysia,6.326672,99.843025
2,660694,Penang Island,Penang,Malaysia,5.388071,100.29352
3,635527,Batu Ferringhi,Penang Island,Malaysia,5.471783,100.24629
4,298303,George Town,Penang Island,Malaysia,5.411938,100.32664
5,298307,Kota Kinabalu,Kota Kinabalu District,Malaysia,5.979383,116.07349
6,303997,Sandakan,Sandakan Division,Malaysia,5.838944,118.11561
7,298283,Langkawi,Langkawi District,Malaysia,6.351074,99.75824
8,298313,Petaling Jaya,Petaling District,Malaysia,3.10363,101.64778
9,666315,Cherating,Kuantan District,Malaysia,4.133792,103.38026


###### Join attr_href_cat_spark_df, tripad_act_spark_df as attr_spark_df

In [49]:
# join dataframe
attr_spark_df = attr_href_cat_spark_df.join(
                    tripad_act_spark_df, attr_href_cat_spark_df.activityId ==
                    tripad_act_spark_df.activityId)\
                .select(
                    attr_href_cat_spark_df.activityId, attr_href_cat_spark_df.locationId,
                    tripad_act_spark_df.name, tripad_act_spark_df.price,
                    tripad_act_spark_df.rating, attr_href_cat_spark_df.category)

In [50]:
# visualize spark dataframe
attr_spark_df.show()

+----------+----------+--------------------+------+------+--------------------+
|activityId|locationId|                name| price|rating|            category|
+----------+----------+--------------------+------+------+--------------------+
|  12962337|    298570|Malaysia Countrys...| 43.27|   4.5|[featured_tours_a...|
|  12905660|    298570|Petronas Twin Tow...|119.49|   4.0|[featured_tours_a...|
|  11807013|    298570|Discover the Best...|142.75|   4.5|[featured_tours_a...|
|  12471375|    298570|Genting Highlands...|103.01|   4.5|[featured_tours_a...|
|  12469580|    298570|Half-Day Kuala Lu...| 43.27|   4.0|[featured_tours_a...|
|  16722683|    298570|KL Hop On Hop Off...| 53.07|   3.0|[featured_tours_a...|
|  11454341|   1096277|Langkawi Archipel...|515.06|   5.0|[featured_tours_a...|
|  11812460|    298570|Private Half-Day ...|377.67|   5.0|[featured_tours_a...|
|  11992275|   1096277|12 Flights: Zipli...|473.86|   5.0|[featured_tours_a...|
|  11991123|    660694|Cheong Fatt Tze G

###### Join tripad_attr_location, attr_spark_df as final_attr_spark_df

In [51]:
# join dataframe
final_attr_spark_df = tripad_loc_spark_df.join(
                    attr_spark_df, tripad_loc_spark_df.locationId ==
                    attr_spark_df.locationId)\
                .select(
                    attr_spark_df.activityId, tripad_loc_spark_df.city, tripad_loc_spark_df.state,
                    tripad_loc_spark_df.country, tripad_loc_spark_df.latitude,
                    tripad_loc_spark_df.longitude, attr_spark_df.name, attr_spark_df.price,attr_spark_df.rating, attr_spark_df.category)

In [53]:
# visualize spark dataframe from pandas
display(final_attr_spark_df.toPandas())

,activityId,city,state,country,latitude,longitude,name,price,rating,category
0,12962337,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Malaysia Countryside and Batu Caves Tour from ...,43.27,4.5,"[featured_tours_and_tickets, luxury_special_oc..."
1,12905660,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Petronas Twin Towers Admission Tickets (E-Tick...,119.49,4.0,"[featured_tours_and_tickets, sightseeing_ticke..."
2,11807013,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Discover the Best Local Food Tour by Night in ...,142.75,4.5,"[featured_tours_and_tickets, private_custom_to..."
3,12471375,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Genting Highlands Day Trip from Kuala Lumpur w...,103.01,4.5,"[featured_tours_and_tickets, luxury_special_oc..."
4,12469580,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Half-Day Kuala Lumpur City Tour,43.27,4.0,"[featured_tours_and_tickets, luxury_special_oc..."
5,16722683,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,KL Hop On Hop Off City Tour,53.07,3.0,"[featured_tours_and_tickets, tours_sightseeing]"
6,11454341,Kuah,Langkawi,Malaysia,6.326672,99.843025,Langkawi Archipelago Jet Ski Tour Including Da...,515.06,5.0,"[featured_tours_and_tickets, water_sports, tou..."
7,11812460,Kuala Lumpur,Wilayah Persekutuan,Malaysia,3.156802,101.69717,Private Half-Day Batu Caves Waterfalls and Hot...,377.67,5.0,"[featured_tours_and_tickets, private_custom_to..."
8,11992275,Kuah,Langkawi,Malaysia,6.326672,99.843025,12 Flights: Zipline Eco Adventure in Langkawi,473.86,5.0,"[featured_tours_and_tickets, outdoor_activities]"
9,11991123,Penang Island,Penang,Malaysia,5.388071,100.29352,Cheong Fatt Tze George Town Penang: The Blue M...,20.61,4.5,"[featured_tours_and_tickets, walking_biking_to..."


In [54]:
# write spark dataframe to parquet
final_attr_spark_df.write.parquet(parquet_path + 'tripad_attr',
                                  mode='overwrite')